In [1]:
# загружаем библиотеки
import pandas as pd
import numpy as np
# убираем предупреждения
import warnings
warnings.filterwarnings('ignore')

In [2]:
# фиксируемся
RANDOM_STATE = 42

In [3]:
# создаем датафрейм для ответов
results_regression = pd.DataFrame(columns = ['model', 'task', 'R2'])
results_classification = pd.DataFrame(columns = ['model', 'task', 'f1', 'accuracy'])

https://www.cs.toronto.edu/~delve/data/boston/bostonDetail.html

In [4]:
# загружаем датасет
data = pd.read_csv('boston.csv')
data.head()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,MEDV
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33,36.2


### 1. Разделите выборку на обучающую и тестовую в отношении 80%/20%, предварительно выделив целевую переменную (колонка 'MEDV').

In [5]:
### Ваш код ###

https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html

In [6]:
# загружаем инструмент
from sklearn.model_selection import train_test_split

In [7]:
# Выделяем feature и target
X = data.drop(['MEDV'], axis=1) # feature
y = data['MEDV'] # target

In [8]:
%%time
# разделяем выборку
X_train, X_test, y_train, y_test= train_test_split(X, y, test_size = 0.2, random_state = RANDOM_STATE)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

CPU times: total: 0 ns
Wall time: 2 ms


((404, 13), (102, 13), (404,), (102,))

### 2. Обучите стандартную регрессию, а также Ridge и  Lasso с параметрами по умолчанию и выведите их R2 на тестовой выборке

In [9]:
### Ваш код ###
r2_lr = 0
r2_ridge = 0
r2_lasso = 0

In [10]:
# загружаем модели и оценку
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.metrics import r2_score

https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html

In [11]:
%%time
# обучение по стандартной регресии
m_lr = LinearRegression()
m_lr.fit(X_train,y_train)
pred_lr = m_lr.predict(X_test)
r2_lr = r2_score(y_test, pred_lr)
r2_lr

CPU times: total: 15.6 ms
Wall time: 19 ms


0.6684825753971645

https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.Ridge.html

In [12]:
%%time
# обучение по регресии с гребневой регуляризацией
m_ridge = Ridge()
m_ridge.fit(X_train,y_train)
pred_ridge = m_ridge.predict(X_test)
r2_ridge = r2_score(y_test, pred_ridge)
r2_ridge

CPU times: total: 0 ns
Wall time: 4 ms


0.6659608075261694

https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.Lasso.html

In [13]:
%%time
# обучение по регресии с лассо регуляризацией
m_lasso = Lasso()
m_lasso.fit(X_train,y_train)
pred_lasso = m_lasso.predict(X_test)
r2_lasso = r2_score(y_test, pred_lasso)
r2_lasso

CPU times: total: 0 ns
Wall time: 6 ms


0.6668687223368214

https://scikit-learn.org/stable/modules/generated/sklearn.metrics.r2_score.html

In [14]:
# фиксируем ответы
results_regression.loc[0] = ['LR', 'task2', r2_lr]
results_regression.loc[1] = ['Ridge', 'task2', r2_ridge]
results_regression.loc[2] = ['Lasso', 'task2', r2_lasso]

### 3. Для Ridge и Lasso подберите коэффициент регуляризации двумя способами 1) GridSearchCV, 2) RidgeCV и LassoCV, в пределах от $10^{-5}$ до $10^5$ (по степеням 10). Посчитайте R2 на тестовой выборке по всем моделям и сравните с предыдущими результатами.

In [15]:
### Ваш код ###
r2_ridge_grid_search = 0
r2_ridge_cv = 0
r2_lasso_grid_search = 0
r2_lasso_cv = 0

In [16]:
# загружаем инструменты
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import RidgeCV, LassoCV

In [17]:
# задаем параметр альфа
parameters = {'alpha': [1e-5, 1e-4, 1e-3, 1e-2, 1e-1, 1, 1e+1, 1e+2, 1e+3, 1e+4, 1e+5]}

https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html

In [18]:
%%time
# GridSearchCV подбираем лучший параметр по регресии с гребневой регуляризацией
ridge_grid_search = GridSearchCV(Ridge(), parameters, scoring='r2').fit(X_train, y_train)
best_alpha = ridge_grid_search.best_params_['alpha']
r2_ridge_grid_search = ridge_grid_search.score(X_test, y_test)
best_alpha, r2_ridge_grid_search

CPU times: total: 234 ms
Wall time: 228 ms


(1e-05, 0.6684825680074258)

In [19]:
%%time
# GridSearchCV подбираем лучший параметр по регресии с лассо регуляризацией
lasso_grid_search = GridSearchCV(Lasso(), parameters, scoring='r2').fit(X_train, y_train)
best_alpha = lasso_grid_search.best_params_['alpha']
r2_lasso_grid_search = lasso_grid_search.score(X_test, y_test)
best_alpha, r2_lasso_grid_search

CPU times: total: 219 ms
Wall time: 217 ms


(1e-05, 0.6684829595885677)

https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.RidgeCV.html

In [20]:
%%time
# RidgeCV подбираем лучший параметр по регресии с гребневой регуляризацией
ridge_cv = RidgeCV(alphas = parameters['alpha']).fit(X_train, y_train)
best_alpha = ridge_cv.alpha_
r2_ridge_cv = ridge_cv.score(X_test, y_test)
best_alpha, r2_ridge_cv

CPU times: total: 15.6 ms
Wall time: 8 ms


(0.01, 0.6684746265859918)

https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LassoCV.html

In [21]:
%%time
# LassoCV подбираем лучший параметр по регресии с лассо регуляризацией
lasso_cv = LassoCV(alphas = parameters['alpha']).fit(X_train, y_train)
best_alpha = lasso_cv.alpha_
r2_lasso_cv = lasso_cv.score(X_test, y_test)
best_alpha, r2_lasso_cv

CPU times: total: 31.2 ms
Wall time: 24 ms


(1e-05, 0.6684829595885677)

In [22]:
# фиксируем ответы
results_regression.loc[3] = ['Ridge_GridSearchCV', 'task3', r2_ridge_grid_search]
results_regression.loc[4] = ['RidgeCV', 'task3', r2_ridge_cv]
results_regression.loc[5] = ['Lasso_GridSearchCV', 'task3', r2_lasso_grid_search]
results_regression.loc[6] = ['LassoCV', 'task3', r2_lasso_cv]

In [23]:
# Вывод: Результат немножко улучшился. RidgeCV и GridSearchCV(Ridge) выявили разные best коэффициенты регуляризации.

### 4. Проведите масштабирование выборки (используйте Pipeline, StandardScaler, MinMaxScaler), посчитайте R2 для Ridge и Lasso с параметрами по умолчанию и сравните с предыдущими результатами.

In [24]:
### Ваш код ###
r2_ridge_standart_scaler = 0
r2_ridge_min_max_scaler = 0
r2_lasso_standart_scaler = 0
r2_lasso_min_max_scaler = 0

In [25]:
# загружаем инструменты
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.pipeline import Pipeline

https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html

https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html

In [26]:
%%time
# создаем конвейер со стандартизацией данных по регресии с гребневой регуляризацией
pipe = Pipeline([('scaler', StandardScaler()), ('clf', Ridge())]).fit(X_train, y_train)
r2_ridge_standart_scaler = pipe.score(X_test, y_test)
r2_ridge_standart_scaler

CPU times: total: 0 ns
Wall time: 7 ms


0.6681901076774437

In [27]:
%%time
# создаем конвейер со стандартизацией данных по регресии с лассо регуляризацией
pipe = Pipeline([('scaler', StandardScaler()), ('clf', Lasso())]).fit(X_train, y_train)
r2_lasso_standart_scaler = pipe.score(X_test, y_test)
r2_lasso_standart_scaler

CPU times: total: 0 ns
Wall time: 7 ms


0.6240447523478461

https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.MinMaxScaler.html

In [28]:
%%time
# создаем конвейер с маштабированием данных по регресии с гребневой регуляризацией
pipe = Pipeline([('scaler', MinMaxScaler()), ('clf', Ridge())]).fit(X_train, y_train)
r2_ridge_min_max_scaler = pipe.score(X_test, y_test)
r2_ridge_min_max_scaler

CPU times: total: 0 ns
Wall time: 6 ms


0.6762207658974605

In [29]:
%%time
# создаем конвейер с маштабированием данных по регресии с лассо регуляризацией
pipe = Pipeline([('scaler', MinMaxScaler()), ('clf', Lasso())]).fit(X_train, y_train)
r2_lasso_min_max_scaler = pipe.score(X_test, y_test)
r2_lasso_min_max_scaler

CPU times: total: 15.6 ms
Wall time: 9 ms


0.2573921442545194

In [30]:
# фиксируем ответы
results_regression.loc[7] = ['Ridge_StandardScaler', 'task4', r2_ridge_standart_scaler]
results_regression.loc[8] = ['Ridge_MinMaxScaler', 'task4', r2_ridge_min_max_scaler]
results_regression.loc[9] = ['Lasso_StandardScaler', 'task4', r2_lasso_standart_scaler]
results_regression.loc[10] = ['Lasso_MinMaxScaler', 'task4', r2_lasso_min_max_scaler]

In [31]:
# Вывод: Лучший результат у Ridge с MinMaxScaler. Lasso с StandardScaler / MinMaxScaler ухудшился.

### 5. Подберите коэффициент регуляризации для Ridge и Lasso на масштабированных данных, посчитайте R2 и сравните с предыдущими результатами.

In [32]:
### Ваш код ###
r2_ridge_standart_scaler_cv = 0
r2_ridge_min_max_scaler_cv = 0
r2_lasso_standart_scaler_cv = 0
r2_lasso_min_max_scaler_cv = 0

In [33]:
%%time
# подбираем лучший параметр по регресии с гребневой регуляризацией со стандартизацией данных
a = 0
for alpha in parameters['alpha']:
        pipe = Pipeline([('scaler', StandardScaler()), ('clf', Ridge(alpha))]).fit(X_train, y_train)
        score = pipe.score(X_test, y_test)
        if score > r2_ridge_standart_scaler_cv:
            r2_ridge_standart_scaler_cv = score
            a = alpha
    
print("best_score - {}".format(r2_ridge_standart_scaler_cv), "alpha - {}".format(a))

best_score - 0.6684825724792514 alpha - 1e-05
CPU times: total: 78.1 ms
Wall time: 66 ms


In [34]:
%%time
# подбираем лучший параметр по регресии с гребневой регуляризацией с маштабированием данных
a = 0
for alpha in parameters['alpha']:
        pipe = Pipeline([('scaler', MinMaxScaler()), ('clf', Ridge(alpha))]).fit(X_train, y_train)
        score = pipe.score(X_test, y_test)
        if score > r2_ridge_min_max_scaler_cv:
            r2_ridge_min_max_scaler_cv = score
            a = alpha
    
print("best_score - {}".format(r2_ridge_min_max_scaler_cv), "alpha - {}".format(a))

best_score - 0.6762207658974605 alpha - 1
CPU times: total: 62.5 ms
Wall time: 52 ms


In [35]:
%%time
# подбираем лучший параметр по регресии с лассо регуляризацией со стандартизацией данных
a = 0
for alpha in parameters['alpha']:
        pipe = Pipeline([('scaler', StandardScaler()), ('clf', Lasso(alpha))]).fit(X_train, y_train)
        score = pipe.score(X_test, y_test)
        if score > r2_lasso_standart_scaler_cv:
            r2_lasso_standart_scaler_cv = score
            a = alpha
    
print("best_score - {}".format(r2_lasso_standart_scaler_cv), "alpha - {}".format(a))

best_score - 0.6684821312777707 alpha - 1e-05
CPU times: total: 62.5 ms
Wall time: 58 ms


In [36]:
%%time
# подбираем лучший параметр по регресии с лассо регуляризацией с маштабированием данных
a = 0
for alpha in parameters['alpha']:
        pipe = Pipeline([('scaler', MinMaxScaler()), ('clf', Lasso(alpha))]).fit(X_train, y_train)
        score = pipe.score(X_test, y_test)
        if score > r2_lasso_min_max_scaler_cv:
            r2_lasso_min_max_scaler_cv = score
            a = alpha
    
print("best_score - {}".format(r2_lasso_min_max_scaler_cv), "alpha - {}".format(a))

best_score - 0.6685732691586148 alpha - 0.001
CPU times: total: 62.5 ms
Wall time: 70.9 ms


In [37]:
# фиксируем ответы
results_regression.loc[11] = ['Ridge_StandardScaler_CV', 'task5', r2_ridge_standart_scaler_cv]
results_regression.loc[12] = ['Ridge_MinMaxScaler_CV', 'task5', r2_ridge_min_max_scaler_cv]
results_regression.loc[13] = ['Lasso_StandardScaler_CV', 'task5', r2_lasso_standart_scaler_cv]
results_regression.loc[14] = ['Lasso_MinMaxScaler_CV', 'task5', r2_lasso_min_max_scaler_cv]

In [38]:
# Вывод: Лучший результат остался таким же, т.к. в Ridge по умолчанию alpha = 1.

### 6. Добавьте попарные произведения признаков и их квадраты (используйте PolynomialFeatures) на масштабированных признаках, посчитайте R2 для Ridge и Lasso с параметрами по умолчанию и сравните с предыдущими результатами.

In [39]:
### Ваш код ###
r2_ridge_standart_scaler_poly = 0
r2_ridge_min_max_scaler_poly = 0
r2_lasso_standart_scaler_poly = 0
r2_lasso_min_max_scaler_poly = 0

In [40]:
# загружаем полиномную функцию
from sklearn.preprocessing import PolynomialFeatures

https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.PolynomialFeatures.html

In [41]:
%%time
# создаем конвейер со стандартизацией данных, попарное произведения признаков и их квадраты
# по регресии с гребневой регуляризацией
p=0
for poly in [PolynomialFeatures(2), PolynomialFeatures(interaction_only=True)]:
              pipe = Pipeline([('scaler', StandardScaler()), ('poly', poly), ('clf', Ridge())]).fit(X_train, y_train)
              score = pipe.score(X_test, y_test)
              if score > r2_ridge_standart_scaler_poly:
                             r2_ridge_standart_scaler_poly = score
                             p = poly

print("best_score - {}".format(r2_ridge_standart_scaler_poly), "poly_model - {}".format(p))

best_score - 0.8485204052230758 poly_model - PolynomialFeatures(interaction_only=True)
CPU times: total: 78.1 ms
Wall time: 23 ms


In [42]:
%%time
# создаем конвейер с маштабированием данных, попарное произведения признаков и их квадраты
# по регресии с гребневой регуляризацией
p=0
for poly in [PolynomialFeatures(2), PolynomialFeatures(interaction_only=True)]:
              pipe = Pipeline([('scaler', MinMaxScaler()), ('poly', poly), ('clf', Ridge())]).fit(X_train, y_train)
              score = pipe.score(X_test, y_test)
              if score > r2_ridge_min_max_scaler_poly:
                             r2_ridge_min_max_scaler_poly = score
                             p = poly

print("best_score - {}".format(r2_ridge_min_max_scaler_poly), "poly_model - {}".format(p))

best_score - 0.8298862467502751 poly_model - PolynomialFeatures()
CPU times: total: 31.2 ms
Wall time: 18 ms


In [43]:
%%time
# создаем конвейер со стандартизацией данных, попарное произведения признаков и их квадраты
# по регресии с лассо регуляризацией
p=0
for poly in [PolynomialFeatures(2), PolynomialFeatures(interaction_only=True)]:
              pipe = Pipeline([('scaler', StandardScaler()), ('poly', poly), ('clf', Lasso())]).fit(X_train, y_train)
              score = pipe.score(X_test, y_test)
              if score > r2_lasso_standart_scaler_poly:
                             r2_lasso_standart_scaler_poly = score
                             p = poly

print("best_score - {}".format(r2_lasso_standart_scaler_poly), "poly_model - {}".format(p))

best_score - 0.7364356228795039 poly_model - PolynomialFeatures(interaction_only=True)
CPU times: total: 15.6 ms
Wall time: 19 ms


In [44]:
%%time
# создаем конвейер с маштабированием данных, попарное произведения признаков и их квадраты
# по регресии с лассо регуляризацией
p=0
for poly in [PolynomialFeatures(2), PolynomialFeatures(interaction_only=True)]:
              pipe = Pipeline([('scaler', MinMaxScaler()), ('poly', poly), ('clf', Lasso())]).fit(X_train, y_train)
              score = pipe.score(X_test, y_test)
              if score > r2_lasso_min_max_scaler_poly:
                             r2_lasso_min_max_scaler_poly = score
                             p = poly

print("best_score - {}".format(r2_lasso_min_max_scaler_poly), "poly_model - {}".format(p))

best_score - 0.2690567084488049 poly_model - PolynomialFeatures(interaction_only=True)
CPU times: total: 15.6 ms
Wall time: 17 ms


In [45]:
# фиксируем ответы
results_regression.loc[15] = ['Ridge_StandardScaler_Poly', 'task6', r2_ridge_standart_scaler_poly]
results_regression.loc[16] = ['Ridge_MinMaxScaler_Poly', 'task6', r2_ridge_min_max_scaler_poly]
results_regression.loc[17] = ['Lasso_StandardScaler_Poly', 'task6', r2_lasso_standart_scaler_poly]
results_regression.loc[18] = ['Lasso_MinMaxScaler_Poly', 'task6', r2_lasso_min_max_scaler_poly]

In [46]:
# Вывод:  Результат улучшился.

### 7. Подберите коэффициент регуляризации для Ridge и Lasso на масштабированных данных, добавив PolynomialFeatures, посчитайте R2 и сравните с предыдущими результатами.

In [47]:
### Ваш код ###
r2_ridge_standart_scaler_poly_cv = 0
r2_ridge_min_max_scaler_poly_cv = 0
r2_lasso_standart_scaler_poly_cv = 0
r2_lasso_min_max_scaler_poly_cv = 0

In [48]:
%%time
# подбираем коэффициент регуляризации для Ridge на стандартизированных данных с полиномной функцией
a = 0
for alpha in parameters['alpha']:
    for poly in [PolynomialFeatures(2), PolynomialFeatures(interaction_only=True)]:
        pipe = Pipeline([('scaler', StandardScaler()), ('clf', Ridge(alpha))]).fit(X_train, y_train)
        score = pipe.score(X_test, y_test)
        if score > r2_ridge_standart_scaler_poly_cv:
            r2_ridge_standart_scaler_poly_cv = score
            a = alpha
    
print("best_score - {}".format(r2_ridge_standart_scaler_poly_cv), "alpha - {}".format(a))

best_score - 0.6684825724792514 alpha - 1e-05
CPU times: total: 109 ms
Wall time: 128 ms


In [49]:
%%time
# подбираем коэффициент регуляризации для Ridge на маштабированныз данных с полиномной функцией
a = 0
for alpha in parameters['alpha']:
    for poly in [PolynomialFeatures(2), PolynomialFeatures(interaction_only=True)]:
        pipe = Pipeline([('scaler', MinMaxScaler()), ('clf', Ridge(alpha))]).fit(X_train, y_train)
        score = pipe.score(X_test, y_test)
        if score > r2_ridge_min_max_scaler_poly_cv:
            r2_ridge_min_max_scaler_poly_cv = score
            a = alpha
    
print("best_score - {}".format(r2_ridge_min_max_scaler_poly_cv), "alpha - {}".format(a))

best_score - 0.6762207658974605 alpha - 1
CPU times: total: 109 ms
Wall time: 106 ms


In [50]:
%%time
# подбираем коэффициент регуляризации для Lasso на стандартизированных данных с полиномной функцией
a = 0
for alpha in parameters['alpha']:
    for poly in [PolynomialFeatures(2), PolynomialFeatures(interaction_only=True)]:
        pipe = Pipeline([('scaler', StandardScaler()), ('clf', Lasso(alpha))]).fit(X_train, y_train)
        score = pipe.score(X_test, y_test)
        if score > r2_lasso_standart_scaler_poly_cv:
            r2_lasso_standart_scaler_poly_cv = score
            a = alpha
    
print("best_score - {}".format(r2_lasso_standart_scaler_poly_cv), "alpha - {}".format(a))

best_score - 0.6684821312777707 alpha - 1e-05
CPU times: total: 141 ms
Wall time: 142 ms


In [51]:
%%time
# подбираем коэффициент регуляризации для Lasso на маштабированныз данных с полиномной функцией
a = 0
for alpha in parameters['alpha']:
    for poly in [PolynomialFeatures(2), PolynomialFeatures(interaction_only=True)]:
        pipe = Pipeline([('scaler', MinMaxScaler()), ('clf', Lasso(alpha))]).fit(X_train, y_train)
        score = pipe.score(X_test, y_test)
        if score > r2_lasso_min_max_scaler_poly_cv:
            r2_lasso_min_max_scaler_poly_cv = score
            a = alpha
    
print("best_score - {}".format(r2_lasso_min_max_scaler_poly_cv), "alpha - {}".format(a))

best_score - 0.6685732691586148 alpha - 0.001
CPU times: total: 156 ms
Wall time: 143 ms


In [52]:
# фиксируем ответы
results_regression.loc[19] = ['Ridge_StandardScaler_Poly_CV', 'task7', r2_ridge_standart_scaler_poly_cv]
results_regression.loc[20] = ['Ridge_MinMaxScaler_Poly_CV', 'task7', r2_ridge_min_max_scaler_poly_cv]
results_regression.loc[21] = ['Lasso_StandardScaler_Poly_CV', 'task7', r2_lasso_standart_scaler_poly_cv]
results_regression.loc[22] = ['Lasso_MinMaxScaler_Poly_CV', 'task7', r2_lasso_min_max_scaler_poly_cv]

### 8. Подберите наилучшую модель (используйте Pipeline, GridSearchSCV) подбирая тип регуляризации (L1,L2), коэффициент регуляризации, метод масштабирования и степень полинома в PolynomialFeatures. Выведите итоговые параметры и результат R2.

In [53]:
### Ваш код ###
# best_params = {}
# print('Параметры лучшей модели:\n', best_params)
# r2_best_model = 0

In [54]:
%%time
# подбираем наилучшую модель
pipe = Pipeline([('scaler', 'passthrough'),('poly', 'passthrough'),('clf', 'passthrough')])
params = [{'scaler': [None, StandardScaler(), MinMaxScaler()],
           'poly': [None, PolynomialFeatures(2), PolynomialFeatures(interaction_only=True)],
           'clf': [Ridge(), Lasso()],
           'clf__alpha': parameters['alpha']}]

search = GridSearchCV(pipe, params, scoring='r2').fit(X_train, y_train)
score = search.score(X_test, y_test)

best_params = search.best_params_
r2_best_model = search.best_params_["clf"]

print('Параметры лучшей модели:\n', best_params)
print('r2_best:', score)

Параметры лучшей модели:
 {'clf': Ridge(alpha=10.0), 'clf__alpha': 10.0, 'poly': PolynomialFeatures(interaction_only=True), 'scaler': StandardScaler()}
r2_best: 0.850028974720067
CPU times: total: 26.7 s
Wall time: 15.6 s


In [55]:
# фиксируем ответ
results_regression.loc[23] = ['Best_Model', 'task8', r2_best_model]

### 9. Разделите выборку на признаки и целевую переменную(колонка class). Замените целевую переменную на числовые значения ('<=50K' - 1, '>50K' - 0).

http://archive.ics.uci.edu/ml/datasets/Adult

In [57]:
# закгружаем датасет
data = pd.read_csv('adult.csv')
data.head()

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,class
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


In [58]:
### Ваш код ###

In [59]:
# Выделяем feature и target
data1 = data.copy() # создаем копию датафрейма
X1 = data1.drop(['class'], axis=1) # feature
y1 = data1['class'] # target
y1[y1 == '<=50K'] = 1 # заменяем целевую переменную на числовые значения (<=50K - 1)
y1[y1 == '>50K'] = 0 # заменяем целевую переменную на числовые значения ('>50K - 0)
y1 = y1.astype(int) # меняем формат на числовой
y1.value_counts() # считаем количесто target по каждому значению (1, 0)

1    37155
0    11687
Name: class, dtype: int64

### 10. Посчитайте метрики accuracy и f1_score на предсказании только самого частого класса в целевой переменной.

In [60]:
### Ваш код ###
f1_most_frequent = 0
acc_most_frequent = 0

https://scikit-learn.org/stable/modules/generated/sklearn.metrics.accuracy_score.html

https://scikit-learn.org/stable/modules/generated/sklearn.metrics.f1_score.html

https://scikit-learn.org/stable/modules/generated/sklearn.dummy.DummyClassifier.html

In [61]:
# загружаем оценки и простой классификатор
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.dummy import DummyClassifier

In [64]:
# посчитаем accuracy аналитически
accuracy = max(y1.value_counts()) / sum(y1.value_counts())
print('Accuracy:', accuracy)

Accuracy: 0.7607182343065395


In [65]:
# посчитаем f1 аналитически
TP = max(y1.value_counts())
FP = min(y1.value_counts())
TN = min(y1.value_counts())
FN = 0
precision = TP / (TP + FP)
recall = TP / (TP + FN)
f1 = 2 * precision * recall / (precision + recall)
print('f1_score:', f1)

f1_score: 0.8640999104619929


In [68]:
# разделяем выборку
X1_train, X1_test, y1_train, y1_test = train_test_split(X1, y1, test_size=0.2, random_state=RANDOM_STATE)
X1_train.shape, X1_test.shape, y1_train.shape, y1_test.shape

((39073, 14), (9769, 14), (39073,), (9769,))

In [70]:
%%time
# посчитаем метрики простым классификатором (используя метод когда всегда возвращается наиболее часто используемая метка класса)
D = DummyClassifier(strategy="most_frequent").fit(X1_train, y1_train)
y_pred = D.predict(X1_test)
acc_most_frequent = accuracy_score(y1_test, y_pred)
f1_most_frequent = f1_score(y1_test, y_pred)
print('Accuracy:', acc_most_frequent)
print('f1_score:', f1_most_frequent)

Accuracy: 0.7589313133381104
f1_score: 0.8629459349356923
CPU times: total: 0 ns
Wall time: 7 ms


In [71]:
# Вывод: метрики посчитанные аналитичести и через простой классификатор совпадают

In [72]:
results_classification.loc[0] = ['Most Frequent class', 'task10', f1_most_frequent, acc_most_frequent]

### 11. Выясните, присутствуют ли в данных пропуски. Если присутствуют, заполните их самыми частыми значениями (испольуйте SimpleImputer)

In [74]:
### Ваш код ###

In [73]:
# посмотрим инфу по датасету
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48842 entries, 0 to 48841
Data columns (total 15 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   age             48842 non-null  int64 
 1   workclass       48842 non-null  object
 2   fnlwgt          48842 non-null  int64 
 3   education       48842 non-null  object
 4   education-num   48842 non-null  int64 
 5   marital-status  48842 non-null  object
 6   occupation      48842 non-null  object
 7   relationship    48842 non-null  object
 8   race            48842 non-null  object
 9   sex             48842 non-null  object
 10  capital-gain    48842 non-null  int64 
 11  capital-loss    48842 non-null  int64 
 12  hours-per-week  48842 non-null  int64 
 13  native-country  48842 non-null  object
 14  class           48842 non-null  object
dtypes: int64(6), object(9)
memory usage: 5.6+ MB


In [75]:
# посчитаем пропущенные значения по каждому признаку
data.isnull().sum()

age               0
workclass         0
fnlwgt            0
education         0
education-num     0
marital-status    0
occupation        0
relationship      0
race              0
sex               0
capital-gain      0
capital-loss      0
hours-per-week    0
native-country    0
class             0
dtype: int64

In [76]:
# посчитаем пропущенные значения датасета
len(data.columns[data.isna().any()].tolist())

0

In [77]:
# Вывод - пропуски отсутствуют.

### 12. Выберите колонки с числовыми и категориальными переменными (используя возможности pandas).

In [78]:
### Ваш код ###

In [79]:
# изучим типы данных признаков
X1 = X1.astype(X1.dtypes)
X1.dtypes

age                int64
workclass         object
fnlwgt             int64
education         object
education-num      int64
marital-status    object
occupation        object
relationship      object
race              object
sex               object
capital-gain       int64
capital-loss       int64
hours-per-week     int64
native-country    object
dtype: object

In [80]:
# выберем колонки с числовыми переменными
n_cols = X1.dtypes[X1.dtypes == 'int64'].index
X1[n_cols]

,age,fnlwgt,education-num,capital-gain,capital-loss,hours-per-week
0,39,77516,13,2174,0,40
1,50,83311,13,0,0,13
2,38,215646,9,0,0,40
3,53,234721,7,0,0,40
4,28,338409,13,0,0,40
...,...,...,...,...,...,...
48837,39,215419,13,0,0,36
48838,64,321403,9,0,0,40
48839,38,374983,13,0,0,50
48840,44,83891,13,5455,0,40


In [81]:
# выберем колонки с категориальными переменными
c_cols =X1.dtypes[X1.dtypes == 'object'].index
X1[c_cols]

,workclass,education,marital-status,occupation,relationship,race,sex,native-country
0,State-gov,Bachelors,Never-married,Adm-clerical,Not-in-family,White,Male,United-States
1,Self-emp-not-inc,Bachelors,Married-civ-spouse,Exec-managerial,Husband,White,Male,United-States
2,Private,HS-grad,Divorced,Handlers-cleaners,Not-in-family,White,Male,United-States
3,Private,11th,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,United-States
4,Private,Bachelors,Married-civ-spouse,Prof-specialty,Wife,Black,Female,Cuba
...,...,...,...,...,...,...,...,...
48837,Private,Bachelors,Divorced,Prof-specialty,Not-in-family,White,Female,United-States
48838,?,HS-grad,Widowed,?,Other-relative,Black,Male,United-States
48839,Private,Bachelors,Married-civ-spouse,Prof-specialty,Husband,White,Male,United-States
48840,Private,Bachelors,Divorced,Adm-clerical,Own-child,Asian-Pac-Islander,Male,United-States


### 13. Создайте пайплайн по обработке числовых и категориальных значений колонок (используйте OneHotEncoder,MinMaxScaler) и посчитайте cross_val_score по алгоритмам LogisticRegression, KNeighborsClassifier, LinearSVC по метрикам accuracy и f1_score.

In [82]:
### Ваш код ###
f1_LR = 0
acc_LR = 0
f1_KNN = 0
acc_KNN = 0
f1_SVM = 0
acc_SVM = 0

https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OneHotEncoder.html

https://scikit-learn.org/stable/modules/generated/sklearn.compose.ColumnTransformer.html

In [86]:
# загрузим кодировщик и трансформер
from sklearn.preprocessing import OneHotEncoder # кодировщик
from sklearn.compose import ColumnTransformer # трансформер колонок

In [87]:
# трансформируем (преобразуем) датафрейм features
n_pr = Pipeline([("scaler", MinMaxScaler())])
c_pr = Pipeline([("oh", OneHotEncoder(handle_unknown="ignore"))])
pr = ColumnTransformer([("c", c_pr, c_cols), ("n", n_pr, n_cols)])
X1_tr = pr.fit_transform(X1)
X1_tr = pd.DataFrame(X1_tr.toarray())
X1_tr

,0,1,2,3,4,5,6,7,8,9,...,98,99,100,101,102,103,104,105,106,107
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.301370,0.044131,0.800000,0.021740,0.0,0.397959
1,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.452055,0.048052,0.800000,0.000000,0.0,0.122449
2,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.287671,0.137581,0.533333,0.000000,0.0,0.397959
3,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.493151,0.150486,0.400000,0.000000,0.0,0.397959
4,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.150685,0.220635,0.800000,0.000000,0.0,0.397959
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48837,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.301370,0.137428,0.800000,0.000000,0.0,0.357143
48838,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.643836,0.209130,0.533333,0.000000,0.0,0.397959
48839,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.287671,0.245379,0.800000,0.000000,0.0,0.500000
48840,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.369863,0.048444,0.800000,0.054551,0.0,0.397959


https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_val_score.html

https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.make_pipeline.html

https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html

https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsClassifier.html

https://scikit-learn.org/stable/modules/generated/sklearn.svm.LinearSVC.html

In [88]:
# загружаем необходимые инструметы и модели
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import LinearSVC

In [90]:
%%time
# создаем пайплайн по обработке числовых и категориальных значений колонок
# считаем cross_val_score по алгоритму LogisticRegression по метрикам accuracy и f1_score
pipeLR = make_pipeline(pr, LogisticRegression())
f1_LR = cross_val_score(pipeLR, X1, y1, scoring='accuracy').mean()
acc_LR = cross_val_score(pipeLR, X1, y1, scoring='f1').mean()
print('acc_LR', acc_LR)
print('f1_LR:', f1_LR)

acc_LR 0.9049854883069461
f1_LR: 0.8511323216589876
CPU times: total: 6.94 s
Wall time: 7.04 s


In [91]:
%%time
# создаем пайплайн по обработке числовых и категориальных значений колонок
# считаем cross_val_score по алгоритму KNeighborsClassifier по метрикам accuracy и f1_score
pipeKNN = make_pipeline(pr, KNeighborsClassifier())
f1_KNN = cross_val_score(pipeKNN, X1, y1, scoring='accuracy').mean()
acc_KNN = cross_val_score(pipeKNN, X1, y1, scoring='f1').mean()
print('acc_KNN', acc_KNN)
print('f1_KNN:', f1_KNN)

acc_KNN 0.8869927593066078
f1_KNN: 0.8247819449093889
CPU times: total: 4min 21s
Wall time: 4min 23s


In [92]:
%%time
# создаем пайплайн по обработке числовых и категориальных значений колонок
# считаем cross_val_score по алгоритму LinearSVC по метрикам accuracy и f1_score
pipeSVM = make_pipeline(pr, LinearSVC())
f1_SVM = cross_val_score(pipeSVM, X1, y1, scoring='accuracy').mean()
acc_SVM = cross_val_score(pipeSVM, X1, y1, scoring='f1').mean()
print('acc_SVM', acc_SVM)
print('f1_SVM:', f1_SVM)

acc_SVM 0.9063223083526415
f1_SVM: 0.8529135478362626
CPU times: total: 13.1 s
Wall time: 13.1 s


In [93]:
# фиксируем ответы
results_classification.loc[1] = ['LogisticRegression', 'task13', f1_LR, acc_LR]
results_classification.loc[2] = ['KNeighborsClassifier', 'task13', f1_KNN, acc_KNN]
results_classification.loc[3] = ['LinearSVC', 'task13', f1_SVM, acc_SVM]

### 14. Можно заметить что в данных присутствуют значения '?', замените их самыми частыми значениями, (испольуйте SimpleImputer). Посчитайте cross_val_score по алгоритмам LogisticRegression, KNeighborsClassifier, LinearSVC по метрикам accuracy и f1_score.

In [94]:
### Ваш код ###
f1_LR = 0
acc_LR = 0
f1_KNN = 0
acc_KNN = 0
f1_SVM = 0
acc_SVM = 0

https://scikit-learn.org/stable/modules/generated/sklearn.impute.SimpleImputer.html

In [95]:
# загружаем простой заполнитель
from sklearn.impute import SimpleImputer

In [96]:
# заменяем значения '?'на их самые частые значения и преобразуем датасет
SI = SimpleImputer(missing_values='?', strategy='most_frequent')
X1_tr_SI = pd.DataFrame(SI.fit_transform(X1), columns=X1.columns).astype(X1.dtypes.to_dict())
X1_tr_SI

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48837,39,Private,215419,Bachelors,13,Divorced,Prof-specialty,Not-in-family,White,Female,0,0,36,United-States
48838,64,Private,321403,HS-grad,9,Widowed,Prof-specialty,Other-relative,Black,Male,0,0,40,United-States
48839,38,Private,374983,Bachelors,13,Married-civ-spouse,Prof-specialty,Husband,White,Male,0,0,50,United-States
48840,44,Private,83891,Bachelors,13,Divorced,Adm-clerical,Own-child,Asian-Pac-Islander,Male,5455,0,40,United-States


In [97]:
# трансформируем (преобразуем) датафрейм features
n_pr = Pipeline([("scaler", MinMaxScaler())])
c_pr = Pipeline([("SI", SI), ("oh", OneHotEncoder(handle_unknown="ignore"))])
pr = ColumnTransformer([("c", c_pr, c_cols), ("n", n_pr, n_cols)])

In [98]:
%%time
# создаем пайплайн по обработке числовых и категориальных значений колонок
# считаем cross_val_score по алгоритму LogisticRegression по метрикам accuracy и f1_score
pipeLR = make_pipeline(pr, LogisticRegression())
f1_LR = cross_val_score(pipeLR, X1, y1, scoring='accuracy').mean()
acc_LR = cross_val_score(pipeLR, X1, y1, scoring='f1').mean()
print('acc_LR', acc_LR)
print('f1_LR:', f1_LR)

acc_LR 0.9048616961932339
f1_LR: 0.85084565041316
CPU times: total: 6.48 s
Wall time: 6.53 s


In [99]:
%%time
# создаем пайплайн по обработке числовых и категориальных значений колонок
# считаем cross_val_score по алгоритму KNeighborsClassifier по метрикам accuracy и f1_score
pipeKNN = make_pipeline(pr, KNeighborsClassifier())
f1_KNN = cross_val_score(pipeKNN, X1, y1, scoring='accuracy').mean()
acc_KNN = cross_val_score(pipeKNN, X1, y1, scoring='f1').mean()
print('acc_KNN', acc_KNN)
print('f1_KNN:', f1_KNN)

acc_KNN 0.8873430482840519
f1_KNN: 0.825252832653795
CPU times: total: 4min 22s
Wall time: 4min 23s


In [100]:
%%time
# создаем пайплайн по обработке числовых и категориальных значений колонок
# считаем cross_val_score по алгоритму LinearSVC по метрикам accuracy и f1_score
pipeSVM = make_pipeline(pr, LinearSVC())
f1_SVM = cross_val_score(pipeSVM, X1, y1, scoring='accuracy').mean()
acc_SVM = cross_val_score(pipeSVM, X1, y1, scoring='f1').mean()
print('acc_SVM', acc_SVM)
print('f1_SVM:', f1_SVM)

acc_SVM 0.9054218744827377
f1_SVM: 0.8512551277675652
CPU times: total: 13.9 s
Wall time: 14 s


In [101]:
# фиксируем ответы
results_classification.loc[4] = ['LogisticRegression_impute', 'task14', f1_LR, acc_LR]
results_classification.loc[5] = ['KNeighborsClassifier_impute', 'task14', f1_KNN, acc_KNN]
results_classification.loc[6] = ['LinearSVC_impute', 'task14', f1_SVM, acc_SVM]

### 15. Посчитайте cross_val_score по тем же алгоритмам и метрикам, если просто удалить значения '?'.

In [102]:
### Ваш код ###
f1_LR_del_missings = 0
acc_LR_del_missings = 0
f1_KNN_del_missings = 0
acc_KNN_del_missings = 0
f1_SVM_del_missings = 0
acc_SVM_del_missings = 0

In [103]:
# Выделяем feature и target
X11 = data.replace('?', np.nan).dropna() # удаляем строки со значениями '?'
X2 = X11.drop(['class'], axis=1) # feature
y2 = X11['class'] # target
y2[y2 == '<=50K'] = 1 # заменяем целевую переменную на числовые значения (<=50K - 1)
y2[y2 == '>50K'] = 0 # заменяем целевую переменную на числовые значения ('>50K - 0)
y2 = y2.astype(int) # меняем формат на числовой
X2.shape, y2.shape

((45222, 14), (45222,))

In [104]:
# выберем колонки с числовыми переменными
n_cols2 = X2.dtypes[X2.dtypes == 'int64'].index
# выберем колонки с категориальными переменными
c_cols2 =X2.dtypes[X2.dtypes == 'object'].index

In [105]:
# трансформируем (преобразуем) датафрейм features
n_pr = Pipeline([("scaler", MinMaxScaler()),])
c_pr = Pipeline([("SI", SI), ("oh", OneHotEncoder(handle_unknown="ignore"))])
pr2 = ColumnTransformer([("c", c_pr, c_cols2), ("n", n_pr, n_cols2)])

In [106]:
%%time
# создаем пайплайн по обработке числовых и категориальных значений колонок
# считаем cross_val_score по алгоритму LogisticRegression по метрикам accuracy и f1_score
pipeLR = make_pipeline(pr2, LogisticRegression())
f1_LR_del_missings = cross_val_score(pipeLR, X2, y2, scoring='accuracy').mean()
acc_LR_del_missings = cross_val_score(pipeLR, X2, y2, scoring='f1').mean()
print('acc_LR', acc_LR_del_missings)
print('f1_LR:', f1_LR_del_missings)

acc_LR 0.90114932100128
f1_LR: 0.8468445746108516
CPU times: total: 6.36 s
Wall time: 6.43 s


In [107]:
%%time
# создаем пайплайн по обработке числовых и категориальных значений колонок
# считаем cross_val_score по алгоритму KNeighborsClassifier по метрикам accuracy и f1_score
pipeKNN = make_pipeline(pr2, KNeighborsClassifier())
f1_KNN_del_missings = cross_val_score(pipeKNN, X2, y2, scoring='accuracy').mean()
acc_KNN_del_missings = cross_val_score(pipeKNN, X2, y2, scoring='f1').mean()
print('acc_KNN', acc_KNN_del_missings)
print('f1_KNN:', f1_KNN_del_missings)

acc_KNN 0.882951165334904
f1_KNN: 0.820574032877531
CPU times: total: 3min 46s
Wall time: 3min 47s


In [108]:
%%time
# создаем пайплайн по обработке числовых и категориальных значений колонок
# считаем cross_val_score по алгоритму LinearSVC по метрикам accuracy и f1_score
pipeSVM = make_pipeline(pr2, LinearSVC())
f1_SVM_del_missings = cross_val_score(pipeSVM, X2, y2, scoring='accuracy').mean()
acc_SVM_del_missings = cross_val_score(pipeSVM, X2, y2, scoring='f1').mean()
print('acc_SVM', acc_SVM_del_missings)
print('f1_SVM:', f1_SVM_del_missings)

acc_SVM 0.9024033119235104
f1_SVM: 0.8485030154158197
CPU times: total: 12.7 s
Wall time: 12.9 s


In [109]:
# фиксируем ответы
results_classification.loc[7] = ['LogisticRegression_delete_missings', 'task15', f1_LR_del_missings, acc_LR_del_missings]
results_classification.loc[8] = ['KNeighborsClassifier_delete_missings', 'task15', f1_KNN_del_missings, acc_KNN_del_missings]
results_classification.loc[9] = ['LinearSVC_delete_missings', 'task15', f1_SVM_del_missings, acc_SVM_del_missings]

 ### 16. Посчитайте cross_val_score для RandomForestClassifier,GradientBoostingClassifier на данных с замененными значениями '?' на самые частые значения.

In [110]:
### Ваш код ###
f1_RF = 0
acc_RF = 0
f1_GB = 0
acc_GB = 0

https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html

https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.GradientBoostingClassifier.html

In [111]:
# выгружаем модели
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier

In [316]:
# трансформируем (преобразуем) датафрейм features
n_pr = Pipeline([("scaler", MinMaxScaler())])
c_pr = Pipeline([("SI", SI), ("oh", OneHotEncoder(handle_unknown="ignore"))])
pr = ColumnTransformer([("c", c_pr, c_cols), ("n", n_pr, n_cols)])

In [112]:
%%time
# считаем cross_val_score для RandomForestClassifier
pipeRF = make_pipeline(pr, RandomForestClassifier())
f1_RF = cross_val_score(pipeRF, X1, y1, scoring='accuracy').mean()
acc_RF = cross_val_score(pipeRF, X1, y1, scoring='f1').mean()
print('acc_RF', acc_RF)
print('f1_RF:', f1_RF)

acc_RF 0.9041934752999119
f1_RF: 0.8509275630705664
CPU times: total: 6min 38s
Wall time: 6min 39s


In [113]:
%%time
# считаем cross_val_score для GradientBoostingClassifier
pipeGB = make_pipeline(pr, GradientBoostingClassifier())
f1_GB = cross_val_score(pipeGB, X1, y1, scoring='accuracy').mean()
acc_GB = cross_val_score(pipeGB, X1, y1, scoring='f1').mean()
print('acc_LR', acc_GB)
print('f1_LR:', f1_GB)

acc_LR 0.9155015819476015
f1_LR: 0.8665903354382216
CPU times: total: 1min 17s
Wall time: 1min 18s


In [114]:
# фиксируем ответы
results_classification.loc[10] = ['RandomForestClassifier', 'task16', f1_RF, acc_RF]
results_classification.loc[11] = ['GradientBoostingClassifier', 'task16', f1_GB, acc_GB]

### 17. Подберите наилучшую модель, подбирая методы обработки колонок - масштабирование признаков, кодирование признаков и заполнение пропусков. Параметры алгоритмов оставьте по умолчанию. Выведите итоговые параметры и результат accuracy и f1_score.

In [115]:
### Ваш код ###
best_params = {}
# print('Параметры лучшей модели:\n', best_params)
f1_best = 0
acc_best = 0

In [127]:
%%time
# выбираем наилучшую модель
scalers = [None, MinMaxScaler(), StandardScaler()]
sis = [None, SimpleImputer(missing_values='?', strategy='most_frequent')]
clfs = [LogisticRegression(), KNeighborsClassifier(), LinearSVC(), RandomForestClassifier(), GradientBoostingClassifier()]

for scaler in scalers:
             for si in sis:    
                            n_pr = Pipeline([("scaler", scaler)])
                            c_pr = Pipeline([("SI", si), ("oh", OneHotEncoder(handle_unknown="ignore"))])
                            pr = ColumnTransformer([("c", c_pr, c_cols), ("n", n_pr, n_cols)])
                            X1_tr = pr.fit_transform(X1)
                            X1_tr = pd.DataFrame(X1_tr.toarray())
                            X1_train, X1_test, y1_train, y1_test= train_test_split(X1_tr, y1, test_size = 0.2,
                                                                                   random_state = RANDOM_STATE)

                            pipe = Pipeline(steps=[('clf', 'passthrough')])
                            params = [{'clf': clfs}]
                            search = GridSearchCV(pipe, params, scoring=('accuracy', 'f1'), refit='f1').fit(X1_train, y1_train)
                            score = search.score(X1_test, y1_test)
            

best_params = search.best_params_
r2_best_model = search.best_params_["clf"]

print('Параметры лучшей модели:\n', best_params)
print('f1_best:', score)

Параметры лучшей модели:
 {'clf': GradientBoostingClassifier()}
f1_best: 0.9152739992226974
CPU times: total: 21min 39s
Wall time: 18min 50s


In [128]:
search.cv_results_

{'mean_fit_time': array([ 1.0450861 ,  0.02249241,  4.06252575,  5.01990304, 16.02426858]),
 'std_fit_time': array([0.0159084 , 0.0067182 , 0.82177345, 1.15708996, 1.25443779]),
 'mean_score_time': array([0.01452088, 7.44924188, 0.00987377, 0.23329401, 0.0246695 ]),
 'std_score_time': array([0.00222278, 0.62495696, 0.00578291, 0.05315112, 0.00883163]),
 'param_clf': masked_array(data=[LogisticRegression(), KNeighborsClassifier(),
                    LinearSVC(), RandomForestClassifier(),
                    GradientBoostingClassifier()],
              mask=[False, False, False, False, False],
        fill_value='?',
             dtype=object),
 'params': [{'clf': LogisticRegression()},
  {'clf': KNeighborsClassifier()},
  {'clf': LinearSVC()},
  {'clf': RandomForestClassifier()},
  {'clf': GradientBoostingClassifier()}],
 'split0_test_accuracy': array([0.84964811, 0.83506078, 0.84990403, 0.85630198, 0.86679463]),
 'split1_test_accuracy': array([0.85476647, 0.84145873, 0.85566219, 0.858

In [133]:
# итоговые параметры и результат accuracy и f1_score
res = pd.DataFrame(search.cv_results_)
f1_best = res['mean_test_accuracy'].max()
acc_best = res['mean_test_f1'].max()
print('acc_best:', acc_best)
print('f1_best:', f1_best)

acc_best: 0.9152649114459679
f1_best: 0.866071006957835


In [134]:
# фиксируем ответ
results_classification.loc[12] = ['Best_Model', 'task17', f1_best, acc_best]

### Результат:

In [56]:
results_regression

,model,task,R2
0,LR,task2,0.668483
1,Ridge,task2,0.665961
2,Lasso,task2,0.666869
3,Ridge_GridSearchCV,task3,0.668483
4,RidgeCV,task3,0.668475
5,Lasso_GridSearchCV,task3,0.668483
6,LassoCV,task3,0.668483
7,Ridge_StandardScaler,task4,0.66819
8,Ridge_MinMaxScaler,task4,0.676221
9,Lasso_StandardScaler,task4,0.624045


In [135]:
results_classification

,model,task,f1,accuracy
0,Most Frequent class,task10,0.862946,0.758931
1,LogisticRegression,task13,0.851132,0.904985
2,KNeighborsClassifier,task13,0.824782,0.886993
3,LinearSVC,task13,0.852914,0.906322
4,LogisticRegression_impute,task14,0.850846,0.904862
5,KNeighborsClassifier_impute,task14,0.825253,0.887343
6,LinearSVC_impute,task14,0.851255,0.905422
7,LogisticRegression_delete_missings,task15,0.846845,0.901149
8,KNeighborsClassifier_delete_missings,task15,0.820574,0.882951
9,LinearSVC_delete_missings,task15,0.848503,0.902403
